In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pickle
from tensorflow import keras
import yaml


import importlib

import Model10_OOP
importlib.reload(Model10_OOP)
from Model10_OOP import ConvModel


import BaseData20Fold
importlib.reload(BaseData20Fold)
from BaseData20Fold import DataProcessor

import Plots
importlib.reload(Plots)
from Plots import plot_CM

# COntains 2stream Neural Network to process fine fatures and coarse features separate


In [5]:
cwd = Path.cwd()

%load_ext tensorboard
%load_ext version_information
#!rm -rf ./logs/

logsdir = cwd / '..' / 'logs'
# !rmdir /s /q {logsdir}
#!tensorboard --logdir {logsdir} --host localhost --port 6006

cfg = 'config_mdl10_concat.yaml'
with open(cfg, 'r') as file:
    config = yaml.safe_load(file)
seeds = config['seeds']

folds = np.arange(1,21)

datapath = cwd / '..' / 'Data' / 'DHG2016' / 'concat_sequences'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


In [12]:

resultsDFlong = pd.DataFrame()
folder = 'concat_sequences'
data_processor = DataProcessor(cfg)
data_processor.load_handgestdata()
data_processor.handangles2windows()
data_processor.save_config(folder)
data_processor.save_windowsets(folder)

#folds = [5]
    
for fold in folds:
    data_processor = DataProcessor(cfg, fold = fold)
    # data_processor.load_handgestdata()
    # data_processor.handangles2windows()
    # data_processor.save_windowsets(folder)
    data_processor.load_windows(folder)
    data_processor.processwindows()
    data_processor.save_windowsets_processed(folder, name='Fold'+str(fold))

# 223 min



permuting Samples using seed 26
Make Windows and apply framreferences...
Processing subject 2
Processing subject 3
Processing subject 4
Processing subject 5
Processing subject 6
Processing subject 7
Processing subject 8
Processing subject 9
Processing subject 10
Processing subject 11
Processing subject 13
Processing subject 14
Processing subject 15
Processing subject 16
Processing subject 17
Processing subject 18
Processing subject 19
Processing subject 20
Using seed 26
Fold: 5
Standardize and padding of data...
Standardize data...train
Write data into windows...
Standardize data...valid
Write data into windows...


In [8]:
with open(cfg, 'r') as file:
    cfgmdl = yaml.safe_load(file)['model']
datapath = cwd / '..' / 'Data' / 'DHG2016' / 'concat_sequences'

results_valid_labels_list = []
results_scores_list_train = []
results_scores_list_valid = []

for seedidx, n_fold in enumerate(folds):
    print('evaluating Fold', n_fold)
    seed_val = seeds[seedidx]
    #for seed_val in seeds:
    fold = pickle.load(open((datapath / ('Fold' + str(n_fold) + '.pkl')).resolve(), 'rb'))
    Y_train_oh = fold['train']['Y_oh']
    Y_valid_oh = fold['valid']['Y_oh']
    X_train = fold['train']['X']
    X_valid = fold['valid']['X']

    like7 = ConvModel(cfg, 'Test', seed = seed_val)
    like7.compile_model()

    # Train the model
    like7.train_model(X_train, Y_train_oh, X_valid, Y_valid_oh)

    Y_prob_train = like7.model.predict(X_train)
    Y_prob_valid = like7.model.predict(X_valid)
    Y_score_true = np.max(Y_prob_valid, axis=-1)

    Y_hat_train = np.argmax(Y_prob_train, axis=-1)
    Y_hat_valid = np.argmax(Y_prob_valid, axis=-1)

    results_train = like7.model.evaluate(X_train, Y_train_oh, verbose = 0, return_dict=True)
    results_valid = like7.model.evaluate(X_valid, Y_valid_oh, verbose = 0, return_dict=True)

    print(f'NLL: {results_valid['NLL']:,.2f}')
    #print(f'Validation score: {results['loss']:,.2f}')
    print(f'Validation accuracy: {results_valid['accuracy']:,.2f}')
    print(f'Validation F1Score: {results_valid['f1_score']:,.2f}')

    results_valid_labels_list.append(pd.DataFrame({'Fold': n_fold, 'Y': fold['valid']['Y'], 'Y_hat': Y_hat_valid, 'Y_prob': Y_score_true}))
    results_scores_list_train.append(pd.DataFrame(results_train, index = [n_fold]))
    results_scores_list_valid.append(pd.DataFrame(results_valid, index = [n_fold]))
    #results_scores_list.append({'Fold': n_fold, 'Accuracy_train': accuracy_train, 'Accuracy_valid': accuracy_valid, 'F1-Score_train': f1_score_train, 'F1-Score_valid': f1_score_valid, 'NLL_train': nll_train, 'NLL_valid': nll_valid})
    
results_scores = pd.concat([pd.concat(results_scores_list_train).add_suffix('_train'),pd.concat(results_scores_list_valid).add_suffix('_valid')],axis = 1)
results_scores.rename(columns = {'fold_train': 'fold'}, inplace = True)
results_valid_labels = pd.concat(results_valid_labels_list)

evaluating Fold 1
Setting seed to 5
Epoch 1/20
1226/1226 - 60s - 49ms/step - NLL: 1.4320 - accuracy: 0.6004 - f1_score: 0.1233 - loss: 1.6652 - precision: 0.7551 - recall: 0.4483 - val_NLL: 1.4207 - val_accuracy: 0.5879 - val_f1_score: 0.1545 - val_loss: 1.5986 - val_precision: 0.7545 - val_recall: 0.4926
Epoch 2/20
1226/1226 - 56s - 45ms/step - NLL: 1.2049 - accuracy: 0.6337 - f1_score: 0.2249 - loss: 1.3852 - precision: 0.7839 - recall: 0.4969 - val_NLL: 1.2598 - val_accuracy: 0.6121 - val_f1_score: 0.2457 - val_loss: 1.4450 - val_precision: 0.7646 - val_recall: 0.5205
Epoch 3/20
1226/1226 - 54s - 44ms/step - NLL: 1.0959 - accuracy: 0.6598 - f1_score: 0.3021 - loss: 1.2855 - precision: 0.7968 - recall: 0.5329 - val_NLL: 1.2411 - val_accuracy: 0.6226 - val_f1_score: 0.2730 - val_loss: 1.4317 - val_precision: 0.7724 - val_recall: 0.5273
Epoch 4/20
1226/1226 - 54s - 44ms/step - NLL: 1.0379 - accuracy: 0.6746 - f1_score: 0.3424 - loss: 1.2306 - precision: 0.8040 - recall: 0.5543 - val_NL

In [10]:
results_valid_labels.to_pickle(datapath/'results_valid_labels_dcnn.pkl')
results_scores.to_pickle(datapath/'results_scores_dcnn.pkl')

In [9]:
mean = results_scores.mean()
std = results_scores.std()

print(f'accuracy on training data - frames: {mean["accuracy_train"]*100:.1f}% +-  {std["accuracy_train"]*100:.1f}%')
print(f'F1_Score on training data - frames: {mean["f1_score_train"]*100:.1f}% +- {std["f1_score_train"]*100:.1f}%')
print(f'NLL on training data - frames: {mean["NLL_train"]:.2f} +- {std["NLL_train"]:.2f}')

print(f'accuracy on validation data - frames: {mean['accuracy_valid']*100:.1f}% +- {std['accuracy_valid']*100:.1f}%')
print(f'F1-Score on validation data - frames: {mean['f1_score_valid']*100:.1f}% +- {std['f1_score_valid']*100:.1f}%')
print(f'NLL on validation data - frames: {mean['NLL_valid']:.2f} +- {std['NLL_valid']:.2f}')

accuracy on training data - frames: 76.0% +-  0.8%
F1_Score on training data - frames: 54.3% +- 2.3%
NLL on training data - frames: 0.73 +- 0.02
accuracy on validation data - frames: 69.4% +- 3.2%
F1-Score on validation data - frames: 39.6% +- 5.5%
NLL on validation data - frames: 1.01 +- 0.12


Ab hier optionaler Code

In [21]:
#DIlatein factor 2, no padding, meanreference, kernelsize2
resultsDFslide

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,5,26,1.641951,0.44632,0.392613


In [18]:
#DIlatein factor 2, no padding, meanreference
resultsDFslide

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,5,26,1.573875,0.453778,0.394649


In [14]:
#DIlatein factor 3, no padding, meanreference
resultsDFslide

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,5,26,1.646613,0.414686,0.400899


In [5]:
#DIlatein factor 3, no padding, firstframereference
resultsDFslide

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,5,26,1.654638,0.435338,0.390421


In [7]:
#DIlatein factor 3, no padding
resultsDFslide

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,5,26,1.923127,0.378135,0.322695


In [10]:
#DIlatein factor 3, causal padding
resultsDFslide

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,5,26,2.011779,0.350352,0.29177


In [16]:
like7.model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 140, 27)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_and_pad_la… │ [(None, 4, 142,   │          0 │ input_layer_1[0]… │
│ (ReshapeAndPadLaye… │ 5), (None, 140,   │            │                   │
│                     │ 7)]               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 1, 140, 6) │        366 │ reshape_and_pad_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 140, 6)    │          0 │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 138, 16)   │        304 │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 138, 16)   │        352 │ reshape_and_pad_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 132, 64)   │      3,136 │ conv1d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 132, 64)   │      3,136 │ conv1d_12[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 114, 64)   │     12,352 │ conv1d_9[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 114, 64)   │     12,352 │ conv1d_13[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 60, 64)    │     12,352 │ conv1d_10[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 60, 64)    │     12,352 │ conv1d_14[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 60, 128)   │          0 │ conv1d_11[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 20, 128)   │          0 │ concatenate_1[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 2560)      │          0 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 200)       │    512,200 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 200)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 15)        │      3,015 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,715,753 (6.55 MB)

 Trainable params: 571,917 (2.18 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,143,836 (4.36 MB)

In [ ]:
%version_information tensorflow, numpy, matplotlib, plotly, pandas, keras